In [1]:
# Cell 1: Imports
import pandas as pd
import numpy as np


In [4]:
# Cell 2: File paths (EDIT THESE TO MATCH YOUR FILE LOCATIONS)
pr_file = "data/pr_hourly_DWD_ID1550.dat"       # file with year, month, day, hour, pr
q_file  = "data/Q_hourly_ID16425004.dat"        # file with year, month, day, hour, Q

# Common options for .dat:
#   sep=r"\s+"  -> whitespace
#   sep=","     -> comma
#   header=None -> if there is no header in the file



In [5]:
# Cell 3: Load the precipitation file (PR)
# Adjust sep= and header= if needed

pr_df = pd.read_csv(
    pr_file,
    sep=r"\s+",      # change if needed
    header=None,     # change to 0 if the file already has column names
)

# If your file has NO header, set column names manually:
# (year, month, day, hour, pr)
pr_df.columns = ["year", "month", "day", "hour", "pr"]

pr_df.head()


,year,month,day,hour,pr
0,1995,9,1,0,0.0
1,1995,9,1,1,0.0
2,1995,9,1,2,0.0
3,1995,9,1,3,0.0
4,1995,9,1,4,0.0


In [8]:
# Cell 4: Load the discharge file (Q)
# Adjust sep= and header= if needed

q_df = pd.read_csv(
    q_file,
    sep=r",",      # change if needed
    header=0,     # change to 0 if the file already has column names
)

# If your file has NO header, set column names manually:
# (year, month, day, hour, Q)
q_df.columns = ["year", "month", "day", "hour", "Q"]

q_df.head()


,year,month,day,hour,Q
0,1920,11,1,0,2.46
1,1920,11,1,1,2.46
2,1920,11,1,2,2.46
3,1920,11,1,3,2.46
4,1920,11,1,4,2.46


In [9]:
# Cell 5: Create a datetime column for both datasets
# Assuming year, month, day, hour are integers and hour is 0–23

for df in [pr_df, q_df]:
    df["datetime"] = pd.to_datetime(
        dict(year=df["year"], month=df["month"], day=df["day"], hour=df["hour"]),
        errors="coerce"  # invalid dates become NaT
    )

# Optionally drop rows with invalid datetime
pr_df = pr_df.dropna(subset=["datetime"])
q_df  = q_df.dropna(subset=["datetime"])

pr_df.head(), q_df.head()


(   year  month  day  hour   pr            datetime
 0  1995      9    1     0  0.0 1995-09-01 00:00:00
 1  1995      9    1     1  0.0 1995-09-01 01:00:00
 2  1995      9    1     2  0.0 1995-09-01 02:00:00
 3  1995      9    1     3  0.0 1995-09-01 03:00:00
 4  1995      9    1     4  0.0 1995-09-01 04:00:00,
    year  month  day  hour     Q            datetime
 0  1920     11    1     0  2.46 1920-11-01 00:00:00
 1  1920     11    1     1  2.46 1920-11-01 01:00:00
 2  1920     11    1     2  2.46 1920-11-01 02:00:00
 3  1920     11    1     3  2.46 1920-11-01 03:00:00
 4  1920     11    1     4  2.46 1920-11-01 04:00:00)

In [10]:
# Cell 6: Check for duplicates by datetime (optional but good practice)

print("PR duplicates by datetime:", pr_df["datetime"].duplicated().sum())
print("Q duplicates by datetime :", q_df["datetime"].duplicated().sum())

# If there are duplicates and you want to keep only the first:
pr_df = pr_df.drop_duplicates(subset="datetime", keep="first")
q_df  = q_df.drop_duplicates(subset="datetime", keep="first")


PR duplicates by datetime: 0
Q duplicates by datetime : 0


In [13]:
def clean_hydro_series(series, var_name="", z_thresh=4, jump_factor=4):
    """
    Best-practice hybrid anomaly cleaner for flood-prediction data.
    """
    s = series.copy().astype(float)

    # 1. Known anomaly flags
    flags = [-9999, -8888, -7777, -5555]
    s = s.replace(flags, np.nan)

    # 2. Physical constraints
    if "pr" in var_name.lower():
        s[s < 0] = np.nan
        s[s > 300] = np.nan   # adjust if tropical climate

    if "q" in var_name.lower():
        s[s < 0] = np.nan

    # 3. Remove statistical outliers
    z = (s - s.mean()) / s.std()
    s[np.abs(z) > z_thresh] = np.nan

    # 4. Remove unrealistic hydrographic jumps
    diff = s.diff().abs()
    median_diff = diff.median()
    s[diff > jump_factor * median_diff] = np.nan

    # 5. Rolling median smoothing
    s = s.rolling(window=12, center=True, min_periods=1).median()

    # 6. Final interpolation
    s = s.interpolate(method="time")

    return s


In [14]:
pr_df = pr_df.sort_values("datetime").set_index("datetime")
q_df  = q_df.sort_values("datetime").set_index("datetime")

pr_df["pr"] = clean_hydro_series(pr_df["pr"], var_name="pr")
q_df["Q"]   = clean_hydro_series(q_df["Q"], var_name="Q")

pr_df = pr_df.reset_index()
q_df = q_df.reset_index()


In [15]:
# Align and merge on datetime
# inner join -> only timestamps that appear in BOTH datasets are kept
# this effectively "removes rows so that the time matches"

merged = pd.merge(
    pr_df[["datetime", "pr"]],
    q_df[["datetime", "Q"]],
    on="datetime",
    how="inner"
)

print("PR shape:", pr_df.shape)
print("Q shape :", q_df.shape)
print("Merged shape:", merged.shape)

merged.head()

PR shape: (213312, 6)
Q shape : (870576, 6)
Merged shape: (213312, 3)


,datetime,pr,Q
0,1995-09-01 00:00:00,0.0,15.339605
1,1995-09-01 01:00:00,0.0,15.305955
2,1995-09-01 02:00:00,0.0,15.272305
3,1995-09-01 03:00:00,0.0,15.238656
4,1995-09-01 04:00:00,0.0,15.205006


In [17]:
# Cell 10: (Optional) Quick check for ML use

# Basic stats
print(merged.describe())

# Check for missing values
merged.isna().sum()


                  datetime             pr              Q
count               213312  213312.000000  213312.000000
mean   2007-10-31 23:30:00       0.041573       4.158630
min    1995-09-01 00:00:00       0.000000       0.226500
25%    2001-09-30 23:45:00       0.000000       1.777720
50%    2007-10-31 23:30:00       0.000000       3.154000
75%    2013-11-30 23:15:00       0.000000       5.398000
max    2019-12-31 23:00:00       2.700000      16.041000
std                    NaN       0.229641       3.307093


datetime    0
pr          0
Q           0
dtype: int64

In [21]:
output_path = "data/merged.csv"
merged.to_csv(output_path, index=False)

output_path


'data/merged.csv'